# Measuring the effect of a sugar tax

[Chapter 3](https://www.core-econ.org/espp/book/text/03.html) of the 
[Economy, Society, and Public Policy](https://www.core-econ.org/espp/index.html)
suggests the project [measuring the effect of a sugar tax](http://www.core-econ.org/doing-economics/book/text/03-01.html) 
to deepen the knowledge of the topic. This notebooks analyses the data provided in the project using Python. In particular this notebook uses the project to
provide a introduction to [pandas](https://pandas.pydata.org/):

> a fast, powerful, flexible and easy to use open source data analysis and manipulation tool.

To run this notebook in Google Colab click on the following batch: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ceedee666/international-teaching-week-2023/blob/main/sugar_tax.ipynb)

The data for this project has already been downloaded and stored in the [dataset_sugar_tax.xlsx](./data/dataset_sugar_tax.xlsx) file located in the [data](./data/) directory. 

### Installation

- Install the required libraries installed by executing `pip3 install pandas openpyxl`
- In an Jupyter Notebook the libraries can be installed using `!pip3 install pandas openpzxl`

In [ ]:
!pip3 install pandas openpyxl

## Loading the dataset

In [pandas](https://pandas.pydata.org/) data is stored in
[`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame)s. A 
[`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame) represents tabular data
like e.g. data from a spreadsheet or a database. 

The `read_excel` function can be used to read data form MS Excel files into
a [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame).

In [ ]:
import pandas as pd

df = pd.read_excel(
    "https://github.com/ceedee666/international-teaching-week-2023/raw/main/data/dataset_sugar_tax.xlsx",
    sheet_name="Data",
)

A pandas [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame) provides different 
methods for giving an overview of the data. One nice feature of Jupyter notebooks is, that 
pandas [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame) are automatically 
shown in a readable format. Compare the result of the following two cells.

In [ ]:
df.head()

In [ ]:
print(df.head())

It is also possible to show just the content of selected columns or just the entries with as certain value. 

In [ ]:
df[["price_per_oz", "price"]]

In [ ]:
df[df["store_id"] == 16]

Using plots it is possible to get an overview of the data in the DataFrame. 

In [ ]:
df["price_per_oz"].plot.hist()

In [ ]:
df.plot.scatter(x="size", y="price")

The `nunique` method can be used to count the individual values in a column.

In [ ]:
num_stores = df["store_id"].nunique()
num_products = df["product_id"].nunique()

print(f"Number of unique stores: {num_stores}")
print(f"Number of unique products: {num_products}")

## Updating the DataFrame 

Most of the data in the DataFrame are numbers. However, some of these numbers represent categorical data. For example,
the value `0` in the `taxed`colum represents beverages that have not been taxed while `1` represents taxed beverages. 

The `map` method can be used to replace the existing values with a textual representation. Note that `map`, like most 
pandas methods, creates a new DataFrame. The result is used to update the existing values. 

In addition it is also possible to create new columns. This is done by simply assigning the result to a new column name. 

In [ ]:
df["taxed"] = df["taxed"].map({0: "not taxed", 1: "taxed"})
df

### Exercise 1
For the different columns the following mappings exist: 

- `supp` column: 
    - `0` represents `Standard`
    - `1` represents `Supplemental`
- `store_type` column: 
    - `1` represents `Large Supermarket`
    - `2` represents `Small Supermarket`
    - `3` represents `Pharmacy`
    - `4` represents `Gas Station`

Replace the values in the `supp` colum by the textual values. Add a new colum named `store_type_str` containing the 
textual representation of the `store_type` values. 

Furthermore, the value `MAR2015` in colum `time` is not correct. The value needs to be changed to `MAR2016`. The `replace` 
method can be used for this purpose. 

In [ ]:
# Implement the update here

## Comparing the treatment and the control group

The goal of this project is to compare two groups before and after introducing the tax:

- treatment group: large supermarkets (`store_type = 1`)
- control group: pharmacies (`store_type = 3`).

A fist step is to summarize the data of treatment and the control group and compare them with respect to the variables of interest. This can be done using the `pivot_table` method. 

### Creating pivot tables

The cell below creates a pivot table `pt_store_type` from the DataFrame `df`. The pivot table uses the `store_type_str` as an index and the `time` as the columns. Each column contains the number of values in the DataFrame. This is done by counting the entries in the `price` column.


In [ ]:
pt_store_type = df.pivot_table(
    index=["store_type_str"],
    columns="time",
    values="price",
    aggfunc="count",
    margins=True,
)
pt_store_type

Note that the rows in the resulting pivot table are sorted alphabetically. Sometimes a other sort order is needed. This can be achieved by re-indexing the DataFrame (a pivot table is also "just" a DataFrame).

In [ ]:
pt_store_type = pt_store_type.reindex(
    ["Large Supermarket", "Small Supermarket", "Pharmacy", "Gas Station", "All"],
)

pt_store_type

It is also possible to use multiple columns in a pivot table. To compare the number of entries for taxed and not taxed beverages depending on the store type the pivot table below can be used.

In [ ]:
pt_store_taxed = df.pivot_table(
    index=["store_type_str"],
    columns=["time", "taxed"],
    values="price",
    aggfunc="count",
    margins=True,
).reindex(
    ["Large Supermarket", "Small Supermarket", "Pharmacy", "Gas Station", "All"],
)
pt_store_taxed

## Exercise 2

Create a pivot table that shows for each product type the number of entries in each period. 

## Calculate and compare conditional means

Besides counting the number of entries in different groups it is also possible to compare other statistical values. The goal of this project is to compare the mean prices of taxed and untaxed beverages, before and after the tax. In the paper underlying this project the following conditions where applied to the data:

- only products that are present in all time periods are included 
- only non-supplementary products (`supp == 0`) are included.

Filtering non-supplementary is quite simple. Pandas provides a nice syntax for selecting only the rows conforming to a certain expression. 

In [ ]:
df = df[df["supp"] != "Supplemental"]
df

Filtering the products that are present in all time periods is more complex. A simplex expression cannot be used to filter the DataFrame. One approach is to perform the following steps.

1. Create a pivot table with the count of all store and product combinations for each time frame.  
1. Filter this pivot table to just retain the rows with at least one entry in each time frame.  
1. Use the rows from the pivot table to filter the DataFrame. 

These steps are executed in the following cells.

In [ ]:
pt_store_product = df.pivot_table(
    index=["store_id", "product_id"], columns="time", values="price", aggfunc="count"
)
pt_store_product

In [ ]:
# select all rows that have values > 0 in all columns
#  1. create a binary mask
#  2. Use this mask for filtering the DataFrame
mask = (pt_store_product > 0).all(axis=1)
pt_store_product = pt_store_product[mask]
pt_store_product

In [ ]:
# filter the data frame
#   1. Set the index of the data frame to the rows used in the pivot table
#   2. Use the loc function to filter the values
filtered_df = df.reset_index()
filtered_df = filtered_df.set_index(["store_id", "product_id"])
filtered_df = filtered_df.loc[pt_store_product.index.values]
filtered_df

Now the mean price of the products can be calculated. 

In [ ]:
mean_prices = filtered_df.pivot_table(
    index=["taxed", "store_type_str"],
    columns="time",
    values="price_per_oz",
    aggfunc="mean",
).round(3)

mean_prices

## Calculate the change in the mean price

In order to compare the groups the next step is to calculate the price changes for each store. 

### Exercise 3

Add two now columns to the `mean_price` DataFrame. The column `d1` contains the price change from `DEC2014` to `JUN2015`. The column `d2` the price change from `DEC2014` to `MAR2016`.  


### Plot column chart of average price change

Finally, a colum chart is used to display the average price change. The cells below create a bar chats comparing 
the price c

In [ ]:
mean_prices_jun15 = mean_prices.reset_index()
mean_prices_jun15 = mean_prices_jun15.pivot(
    index="store_type_str", columns="taxed", values="d1"
)
mean_prices_jun15.plot.bar()

In [ ]:
mean_prices_mar16 = mean_prices.reset_index()
mean_prices_mar16 = mean_prices_mar16.pivot(
    index="store_type_str", columns="taxed", values="d1"
)
ax = mean_prices_mar16.plot.bar()
ax.set_title("Price Change from 12/2014 to 03/2016")
ax.set_xlabel("Store Type")
ax.set_ylabel("Price Change per Oz")

## References

- [pandas Documentation](https://pandas.pydata.org/docs/)
- [pandas Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)